# ISCE Processing of SnowEx UAVSAR data
By: **Jack Tarricone** *(University of Nevada, Reno)* \
Guidance from: **Sam Neitlich** *(Montana State/JPL)*, **Yungling Lou** *(JPL)*, **Zach Keskenin** *(Boise State)*, **Ross Palomaki** *(Montana State)*, **Naheem Adebisi** *(Boise State)*

In [1]:
import os
import wget
import glob
import xarray as xr
import rioxarray as rxr
import numpy as np
import uavsar_pytools
import matplotlib.pyplot as plt
import matplotlib
import rasterio as rio
from osgeo import gdal
from rasterio.plot import show

ModuleNotFoundError: No module named 'wget'

This tutorial is a recreation of the processing steps outlined [IsceUAVSARprocessing.ipynb](https://github.com/taliboliver/Isce_UAVSAR/blob/master/IsceUAVSARprocessing.ipynb), with addtional functionality added by the ```uavsar_pytools``` package. It is assumed that users have ISCE installed on thier local machines, which admittedly can be the most challenging step in this process.  For information on ISCE installoation please see the [Isce_UAVSAR](https://github.com/taliboliver/Isce_UAVSAR) by **Talib Oliver** *(JPL)* or Eric fieldings (find link from Zach)

In this notebook, we will interfermoterically process SLC stack from the Jemez Mountains, NM using the [Interferometric synthetic aperture radar Scientific Computing Environment (ISCE2)](https://github.com/isce-framework/isce2). We will then phase unwrap using [SNAPHU](https://web.stanford.edu/group/radar/softwareandlinks/sw/snaphu/), and geolocate the SLC data using ```geolocate_uavsar()```.

Past tutorials have assumed that your Jupyter environment will natively run command line code. In this notebook will run ISCE using python functionality.

## Create folder to work out of
First we'll create a new folder in our home directory to download files and run ISCE from

In [ ]:
# get home dir
home = os.getcwd()
print(home)

In [ ]:
# working folder name
isce_directory = 'jemez_isce'
    
# path to folder
path = os.path.join(home,isce_directory)

# make new folder
os.mkdir(path)
print("Directory '%s' created" %isce_directory)

In [ ]:
# nativate to new folder
os.chdir(path)
os.getcwd()

In [ ]:
# working folder name
isce_directory = 'jemez_isce'
    
# path to folder
path = os.path.join(home,isce_directory)

# make new folder
os.mkdir(path)

## Data Download

Data links can be found on the UAVSAR site. In this case we'll be using SLCs from Feb 12th and 26th
![](/Users/jacktarricone/uavsar_snow/examples/imgs/uavsar_data_portal.png)



In [ ]:
# create download folder
download = 'download'
dl = os.path.join(path,download)
os.mkdir(dl)

In [ ]:
os.chdir(dl)
os.getcwd()

Here the links are formatted as a string, the ```wget``` removed, and saved as a text file. This can also be down outside of this notebook by copying the links directly from the UAVSAR data portal and saving into your local text editor.

In [ ]:
links = ['https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_01_BU.dop',
         'https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_01_BU_s1_2x8.llh',
         'https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_01_BU_s1_2x8.lkv',
         'https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_20005_003_200212_L090HH_01_BU_s1_2x8.slc',
         'https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_20005_003_200212_L090HH_01_BU.ann',
         'https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_20013_000_200226_L090HH_01_BU_s1_2x8.slc',
         'https://downloaduav2.jpl.nasa.gov/Release30/alamos_35915_01/alamos_35915_20013_000_200226_L090HH_01_BU.ann']

with open('downloads.txt', 'w') as f:
    for links in links:
        f.write(links)
        f.write('\n')

In [ ]:
type(links)

In [ ]:
import wget
'http://www.futurecrew.com/skaven/song_files/mp3/razorback.mp3'
filename = wget.download(url)